In [1]:
import pandas as pd
import numpy as np
import pickle, time, math

In [2]:
data = pd.read_csv('train.csv',delimiter=',')
#dt2 = pd.read_csv('test.csv',delimiter=',')

C:\Users\h_agu\Desktop\machine_learning\Miniconda3\envs\tensorflow-gpu3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#"not available in demo dataset","na"
data=data.replace({"not available in demo dataset":"(nad)"},regex=True) # nad = not avail for demo
data=data.replace({'(not set)':"(ns)"},regex=True) # ns = not set
data=data.replace({'(not provided)':'(np)'},regex=True) # np = not provided

In [4]:
cols=['device','geoNetwork','totals','trafficSource']

for col in cols:
    df1 = pd.DataFrame(data.pop(col).apply(pd.io.json.loads).values.tolist(), index=data.index)
    df1.columns = [ str(col) + '_' + str(Col) for Col in df1.columns]
    data=data.join(df1)

In [5]:
col='trafficSource_adwordsClickInfo'
columns=[]
dat_dict=dict()
for idx,dat in zip(data.index,data[col]):
    #print(idx)
    for key,d in dat.items():
        #a=d.split(':')
        if key not in columns:
            columns.append(key)
            dat_dict.update({key:{'idx':[idx],'val':[d]}})
        else:
            _dict=dat_dict[key]
            _idx=_dict['idx']
            _idx.append(idx)
            _val=_dict['val']
            _val.append(d)

for _col,dat in dat_dict.items():
    df1=pd.DataFrame(dat['val'],index=dat['idx'],columns=[col +'_'+_col.replace("'",'')])
    #print(df1.head())
    data=data.join(df1,how='outer')
    
data=data.drop([col],axis=1)

In [6]:
col_to_drop=['device_browserSize', 'device_browserVersion', 'device_flashVersion', 'device_language'
, 'device_mobileDeviceBranding', 'device_mobileDeviceInfo', 'device_mobileDeviceMarketingName'
, 'device_mobileDeviceModel', 'device_mobileInputSelector', 'geoNetwork_latitude', 'geoNetwork_longitude'
, 'geoNetwork_networkLocation', 'trafficSource_campaignCode', 'socialEngagementType', 'device_operatingSystemVersion'
, 'device_screenColors', 'device_screenResolution', 'trafficSource_adwordsClickInfo_criteriaParameters'
, 'trafficSource_adwordsClickInfo_isVideoAd', 'trafficSource_adwordsClickInfo_targetingCriteria', 'trafficSource_isTrueDirect'
, 'geoNetwork_cityId'
#, 'trafficSource_adwordsClickInfo_targetingCriteria_imputed', 'totals_transactionRevenue_imputed'
]
data=data.drop(col_to_drop,axis=1)

In [7]:
col_num=['totals_bounces', 'totals_hits', 'totals_newVisits', 'totals_pageviews']
col_app=[]
for col in col_num:
    data[col +"_imputed"]=pd.isnull(data[col]).astype(np.int)
    col_app.append(col +"_imputed")
    data[col].fillna(0,inplace=True)
    
drop_imputed=[]
for col in col_app:
    n=(data[col][data[col]==1]).shape[0]
    if n == 0:
        drop_imputed.append(col)
data=data.drop(drop_imputed,axis=1)  

In [8]:
data['totals_bounces']=data['totals_bounces'].astype(np.int)
data['totals_hits']=data['totals_hits'].astype(np.int)
data['totals_newVisits']=data['totals_newVisits'].astype(np.int)
data['totals_pageviews']=data['totals_pageviews'].astype(np.int)
data['totals_transactionRevenue'].fillna(0,inplace=True)
data['totals_transactionRevenue']=data['totals_transactionRevenue'].astype(np.float)
data=data.drop(['totals_visits'],axis=1)

In [9]:
d=data.dtypes.T.apply(pd.Series)
for col, ty in zip(d.index,d.values):
    if ty == 'object' :
        data[col]=data[col].replace(np.nan,'?')

In [10]:
data['totals_transactionRevenue'].fillna(0,inplace=True)

In [ ]:
bins=[5,10,30,50,100]
column_name='totals_pageviews'
#bin_labels=['visit_1to5','visit_5to10','visit_10to30','visit_30to50','visit_50to100','visit_100to150']
#_=[for _bin in bins]
#print(len(bins),len(bin_labels))
bin_labels=[]
for i,_bin in enumerate(bins):
    if i == 0 :
        bin_labels.append(column_name+'_to_'+str(_bin))
        data[bin_labels[i]]=data[column_name]<=_bin
        data[bin_labels[i]].astype(np.bool)
    else :
        bin_labels.append(column_name+'_between_'+str(bins[i-1])+'_'+str(_bin))
        data[bin_labels[i]]=(data[column_name]<=_bin) & (data[column_name]>bins[i-1])
        data[bin_labels[i]].astype(np.bool)
    if len(bins)-1 == i:
        bin_labels.append(column_name+'_greater_'+str(_bin))
        data[bin_labels[i+1]]=(data[column_name]>_bin)
        data[bin_labels[i+1]].astype(np.bool)

In [ ]:
bins=[5,10,30,50,100]
column_name='totals_hits'
#bin_labels=['visit_1to5','visit_5to10','visit_10to30','visit_30to50','visit_50to100','visit_100to150']
#_=[for _bin in bins]
#print(len(bins),len(bin_labels))
bin_labels=[]
for i,_bin in enumerate(bins):
    if i == 0 :
        bin_labels.append(column_name+'_to_'+str(_bin))
        data[bin_labels[i]]=data[column_name]<=_bin
        data[bin_labels[i]].astype(np.bool)
    else :
        bin_labels.append(column_name+'_between_'+str(bins[i-1])+'_'+str(_bin))
        data[bin_labels[i]]=(data[column_name]<=_bin) & (data[column_name]>bins[i-1])
        data[bin_labels[i]].astype(np.bool)
    if len(bins)-1 == i:
        bin_labels.append(column_name+'_greater_'+str(_bin))
        data[bin_labels[i+1]]=(data[column_name]>_bin)
        data[bin_labels[i+1]].astype(np.bool)

In [11]:
y=data['totals_transactionRevenue']
x=data.copy()
x.drop(['totals_transactionRevenue'],axis=1,inplace=True)
y=pd.DataFrame(y,columns=['totals_transactionRevenue'], index=x.index)

In [13]:
y.to_csv('y.csv',index=False)
x.to_csv('x.csv',index=False)

In [14]:
x=pd.read_csv('x.csv')
y=pd.read_csv('y.csv')

C:\Users\h_agu\Desktop\machine_learning\Miniconda3\envs\tensorflow-gpu3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
def categories_encoder_dictionary(x,y,categories_cols):
    inflection=2
    steepness=0.25
    idx=y[y['totals.transactionRevenue']>0].index
    n_rec=y.shape[0]
    n_rec_event=idx.shape[0]
    n_rec_non_event=n_rec-n_rec_event  
    cat_dict=dict()
    zero=1e-6
    for col in categories_cols:
        g1=x.groupby([col])['date'].count()#],columns=['channelGrouping','instances'])
        g2=x.loc[idx].groupby([col])['date'].count()
        g1=pd.DataFrame(g1)
        g1.columns=['instance']
        g2=pd.DataFrame(g2)
        g2.columns=['events']
        g1=g1.join(g2)
        g1['events']=g1.events.fillna(0)
        g1['non_event']=g1['instance']-g1['events']
        g1['prec_non_event']=(g1['non_event']/n_rec_non_event)*100
        g1['prec_event']=(g1['events']/n_rec_event)*100
        g1['WoE']=np.log(((g1['non_event']+zero)/n_rec_non_event)/((g1['events']+zero)/n_rec_event))
        g1['Info_val']=(g1['prec_non_event']-g1['prec_event'])/100*g1['WoE']
        g1['lambda']=1/(1+np.exp(-(g1['events']-inflection)/steepness))
        g1['Target_Mean']=g1['lambda']*g1['events']/g1['instance']+(1-g1['lambda'])*n_rec_event/n_rec
        cat_dict.update({col:g1})
    return cat_dict

In [40]:
data['visitStartTime_dayofweek']=data['visitStartTime'].dt.dayofweek
data['visitStartTime_dayofyear']=data['visitStartTime'].dt.dayofyear

In [73]:
data['visitStartTime']=pd.to_datetime(data['visitStartTime'])
data['visitStartTime_hourofday']=data['visitStartTime'].dt.hour

In [85]:
data['visitStartTime']=pd.to_datetime(data['visitStartTime'])
data['visitStartTime_dayofmonth']=data['visitStartTime'].dt.day

In [84]:
data=data.drop(['visitStartTime_dayofmont'],axis=1)

In [86]:
data['visitStartTime']=pd.to_datetime(data['visitStartTime'])
data['visitStartTime_month']=data['visitStartTime'].dt.month

In [87]:
data['visitStartTime']=pd.to_datetime(data['visitStartTime'])
data['visitStartTime_year']=data['visitStartTime'].dt.year

In [88]:
data['date']=pd.to_datetime(data['date'])
data['date_day']=data['date'].dt.day
data['date_month']=data['date'].dt.month
data['date_year']=data['date'].dt.year
data['date_dayofyear']=data['date'].dt.dayofyear
data['date_dayofweek']=data['date'].dt.dayofweek

In [97]:
data['date']=pd.to_datetime(data['date'])
data['date_quarter']=data['date'].dt.quarter
data['visitStartTime']=pd.to_datetime(data['visitStartTime'])
data['visitStartTime_quarter']=data['visitStartTime'].dt.quarter

In [98]:
data=data.sort_index(axis=1)

In [99]:
data.to_csv('train_data.csv',index=False)

In [100]:
data=pd.read_csv('train_data.csv')

C:\Users\h_agu\Desktop\machine_learning\Miniconda3\envs\tensorflow-gpu3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [101]:
data.dtypes.T

channelGrouping                                  object
date                                             object
date_day                                          int64
date_dayofweek                                    int64
date_dayofyear                                    int64
date_month                                        int64
date_quarter                                      int64
date_year                                         int64
device_browser                                   object
device_deviceCategory                            object
device_isMobile                                    bool
device_operatingSystem                           object
fullVisitorId                                    object
geoNetwork_city                                  object
geoNetwork_city_is_revenue_above_mean              bool
geoNetwork_city_trans_view_above_1_per_mile        bool
geoNetwork_continent                             object
geoNetwork_country                              

to_drop=['visitStartTime','visitNumber','visitId','totals_hits','totals_pageviews','sessionId','fullVisitorId,'date']